<a href="https://colab.research.google.com/github/JairEsc/Mat_Apl_2/blob/main/Epid_AFB_MCMC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
from scipy.integrate import odeint

In [7]:
Q=2000
alpha=0.25
sigma=0.75
mu=1/26.5
tau=1e-4
c=.5
true_params = np.array((Q, alpha,sigma,mu,tau,c))
def AFB(y,t,ps):
    try:
        Q = ps['Q'].value
        alpha = ps['alpha'].value
        sigma = ps['sigma'].value
        mu = ps['mu'].value
        tau = ps['tau'].value
        c = ps['c'].value
    except:
        Q,alpha,sigma,mu,tau,c = ps
    L_s,L_i,B=y
    L_s_p=Q*(L_s+B)/(L_i+L_s+B)-L_s*(alpha-sigma*(B/(L_i+L_s+B)))-L_s*B*tau*(L_i/(L_i+L_s))
    L_i_p=L_s*B*tau*(L_i/(L_i+L_s))-c*L_i*(B/(L_i+L_s+B))
    B_p=L_s*(alpha-sigma*(B/(L_i+L_s+B)))-mu*B
    if(L_i==0):
        L_i_p=0
    return np.array([L_s_p,L_i_p,B_p])
L_s_fijo=(1/(2*alpha))*((Q/mu)*((-alpha+sigma+mu)+np.sqrt((alpha-sigma-mu)**2+4*alpha*mu)))
L_i_fijo=0
B_fijo=Q/mu

In [12]:
y0 = [100,1,20]
plt.figure(figsize=(10, 8))
INPUT = (y0[0],y0[1],y0[2])
t = np.linspace(0, 119, 119*7+1)
from scipy.integrate import odeint
sol = odeint(AFB, y0, t, args=([Q,alpha,sigma,mu,tau,c],))

<Figure size 720x576 with 0 Axes>

In [10]:
from scipy import stats

In [13]:
frecuancia_muestra=np.arange(0,len(t),7*7)
np.random.seed(0)
datos_larvas_infectados=np.array([stats.binom.rvs(n=int(sol[i,1]),p=0.85) for i in range(len(t))])[frecuancia_muestra]
datos_larvas_sanas=np.array([stats.binom.rvs(n=int(sol[i,0]),p=0.80) for i in range(len(t))])[frecuancia_muestra]
data=np.array([datos_larvas_sanas,datos_larvas_infectados]).T


In [16]:
data[:,1]

array([    1,     1,    19,  3576, 12509,  5898,  5026,  5426,  5707,
        5702,  5622,  5597,  5581,  5658,  5660,  5632,  5639,  5622])

In [ ]:
L_S_0=100
L_I_0=1
B_0=20
X0=[L_S_0,L_I_0,0,B_0]
INPUT=[L_S_0,L_I_0,0,B_0]

In [18]:
#### verosimilitud
def logverosimilitud(theta):
    Q, alpha,sigma,mu,tau,c = theta
    dias = 119
    t = np.linspace(0,119,120)  ## vector de tiempos
    Xt =  odeint(func=AFB, y0=X0, t=t, args=(Q, alpha,sigma,mu,tau,c))
    ### Incidencia
    #Inc = np.diff(Xt[:,3])#Valores de incidencia diaria.
    vers = scipy.stats.poisson.logpmf(data[:,1], Xt[:,1])#Suponiendo que dado theta, los datos siguen distribucion 
            #Poisson de parámetro dado por el valor obtenido en la solución de la ODE
    ver = np.sum(vers)
    return(ver)
#### Posterior (Es más fácil trabajar con -log(Posterior))
par1=[0.5,0,1]  #hiperparametros para beta
par2=[0.5,0,1]  #hiperparametros para lamba
par3=[0.5,0,1]  #hiperparametros para gamma
par4=[0.5,0,1]  #hiperparametros para gamma
par5=[0.5,0,1]  #hiperparametros para gamma
par6=[0.5,0,1]  #hiperparametros para gamma
def logaprioris(theta):
    Q, alpha,sigma,mu,tau,c = theta
    dQ = scipy.stats.gamma.logpdf(Q,par1[0],par1[1],par1[2])
    dalpha = scipy.stats.gamma.logpdf(alpha,par2[0],par2[1],par2[2])
    dsigma = scipy.stats.gamma.logpdf(sigma,par3[0],par3[1],par3[2])
    dmu = scipy.stats.gamma.logpdf(mu,par4[0],par4[1],par4[2])
    dtau = scipy.stats.gamma.logpdf(tau,par5[0],par5[1],par5[2])
    dc = scipy.stats.gamma.logpdf(c,par6[0],par6[1],par6[2])
    return(dQ+dalpha+dsigma+dmu+dtau+dc)
def minuslogposterior(theta):
  return(-(logaprioris(theta)+logverosimilitud(theta)))


In [ ]:
!pip install lmfit  #Solo correr una vez

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 292 kB 4.1 MB/s 
     |████████████████████████████████| 98 kB 6.7 MB/s 
  Created wheel for lmfit: filename=lmfit-1.0.3-py3-none-any.whl size=84413 sha256=f11faf4c3ff72b4f9ba27960b37c08b9adecb75208197d1855837719f1ecb85a
  Stored in directory: /root/.cache/pip/wheels/b9/7a/d1/236aa0f8196b264fda481a112f7cfb1bfde7bfb20235f8e331
Successfully built lmfit


In [ ]:
import pytwalk#Añadir el archivo .py a los archivos de colab

In [ ]:
np.random.seed(1)
t=np.arange( 0,18+1,1 )
t0=t[0:119]
I0=1
Y0=1
X0=np.array( X0 )#
T=int(1e5) #iterations
bi=int(0.15* T)#calentamiento
Q_0, alpha_0,sigma_0,mu_0,tau_0,c_0=
tchain = pytwalk.pytwalk( n=3, U=minuslogposterior, )
tchain.Run( T=T , x0=np.array([Q_0, alpha_0,sigma_0,mu_0,tau_0,c_0]), xp0=xp0)

tchain.Ana()
tchain.IAT()